# Hyperliquid API Exploration

This notebook is dedicated to exploring the Hyperliquid API and its functionalities. It allows users to interact with the API and analyze the data it provides. Different to Dune in that its based on RESTAPI.

In [23]:
import sys
import os
import pandas as pd
from datetime import datetime

# Add project root to Python path
project_root = os.path.abspath('..')  # Go up one level from notebooks/
sys.path.insert(0, project_root)

print(f"Added to Python path: {project_root}")
print(f"Current working directory: {os.getcwd()}")

from src.data_providers.factory import setup_providers
from datetime import datetime

# Setup providers
manager = setup_providers()

# Test connections
print("Connection Status:")
for provider, status in manager.test_all_connections().items():
    print(f"  {provider}: {'✅ Connected' if status else '❌ Failed'}")

# Get Hyperliquid provider from the manager
hl = manager.get_provider('hyperliquid')

if hl:
    print("\n🚀 Testing Hyperliquid API endpoints:")
    
    # 1. Get available symbols
    symbols = hl.get_available_symbols()
    print(f"📊 Available symbols: {symbols[:10]}...")  # Show first 10
    
    # 2. Get current mid prices
    mids = hl.get_all_mids()
    print(f"\n💰 Current mid prices:")
    print(mids.head())
    
    # 3. Get market data for ETH
    eth_data = hl.get_market_data('ETH', interval='1h', lookback_hours=48)
    print(f"\n📈 ETH OHLCV data (last 48h):")
    print(f"Shape: {eth_data.shape}")
    print(eth_data.tail())
    
    # 4. Get funding rates
    funding = hl.get_funding_rates()
    print(f"\n💸 Funding rates:")
    print(funding.head())
    
    # 5. Save data for analysis
    if not eth_data.empty:
        data_dir = os.path.join(project_root, 'data', 'raw', 'hyperliquid')
        os.makedirs(data_dir, exist_ok=True)
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        file_path = os.path.join(data_dir, f'eth_ohlcv_{timestamp}.parquet')
        
        eth_data.to_parquet(file_path)
        print(f"\n✅ Data saved: {file_path}")

else:
    print("❌ Hyperliquid provider not available")

2025-09-10 21:31:06,522 INFO DuneProvider Fetching fresh data for query 5745512


Added to Python path: /Users/Max/Code/Github/ML_driven_on-chain_metrics
Current working directory: /Users/Max/Code/Github/ML_driven_on-chain_metrics/notebooks
✅ Dune provider initialized
✅ Hyperliquid provider initialized
Connection Status:
  dune: ✅ Connected
  hyperliquid: ✅ Connected

🚀 Testing Hyperliquid API endpoints:
📊 Available symbols: ['BTC', 'ETH', 'ATOM', 'MATIC', 'DYDX', 'SOL', 'AVAX', 'BNB', 'APE', 'OP']...

💰 Current mid prices:
  symbol  mid_price                  timestamp
0     @1  18.609000 2025-09-10 21:31:07.795908
1    @10   0.000134 2025-09-10 21:31:07.795948
2   @100   0.003523 2025-09-10 21:31:07.795957
3   @101   0.309430 2025-09-10 21:31:07.796186
4   @102   0.018979 2025-09-10 21:31:07.796194


2025-09-10 21:31:08,420 ERROR HyperliquidProvider Failed to get market data for ETH: Length mismatch: Expected axis has 10 elements, new values have 6 elements



📈 ETH OHLCV data (last 48h):
Shape: (0, 0)
Empty DataFrame
Columns: []
Index: []

💸 Funding rates:
    symbol  funding_rate                  timestamp
0  unknown  1.250000e-05 2025-09-10 21:31:09.368454
1  unknown  1.250000e-05 2025-09-10 21:31:09.368483
2  unknown  1.250000e-05 2025-09-10 21:31:09.368487
3  unknown  0.000000e+00 2025-09-10 21:31:09.368489
4  unknown  4.937000e-07 2025-09-10 21:31:09.368490


## Data Exploration

Once the data is fetched, we can explore its structure and contents.

In [26]:
# Standard setup for ALL notebooks
from src.data_providers.factory import setup_providers

# One line setup
manager = setup_providers()

# Quick comprehensive analysis
results = manager.run_comprehensive_analysis(
    symbols=['HYPE', 'ETH', 'BTC'],
    days_back=30,
    save_data=True
)

# Access individual providers if needed
hl = manager.get_provider('hyperliquid')
dune = manager.get_provider('dune')

✅ Dune provider initialized
✅ Hyperliquid provider initialized


AttributeError: 'MultiProviderManager' object has no attribute 'run_comprehensive_analysis'

🔍 Debugging Provider Instantiation
1️⃣ Factory Provider:
   Type: <class 'src.data_providers.hyperliquid.HyperliquidProvider'>
   Has validate_connection: True
   Has get_market_data: True
   Module: src.data_providers.hyperliquid


2025-09-10 21:33:01,472 ERROR HyperliquidProvider Failed to get market data for ETH: Length mismatch: Expected axis has 10 elements, new values have 6 elements


   Test call result: Empty

2️⃣ Direct Provider:
   Type: <class 'src.data_providers.hyperliquid.HyperliquidProvider'>
   Has validate_connection: True
   Has get_market_data: True
   Module: src.data_providers.hyperliquid


2025-09-10 21:33:01,751 ERROR HyperliquidProvider Failed to get market data for ETH: Length mismatch: Expected axis has 10 elements, new values have 6 elements


   Test call result: Empty

3️⃣ Same instance? False
Same class? True


## Further Analysis

You can add additional cells here to perform further analysis on the fetched data, such as visualizations or statistical summaries.